In [18]:
%matplotlib inline

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 

import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display,clear_output

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")


from src.network_sim import NetworkSimulator
from src.algorithm import stochastic_wmmse
from src.experiment import ExperimentInterferenceModelComparison

%reload_ext autoreload
%autoreload 2
%aimport src

def plot_convergence(data, ax):

    data_grouped = data.groupby('iteration').agg(['mean', 'std', 'count'])
    column_names = data_grouped.droplevel(axis=1, level=1).columns.unique().tolist()
    for col in column_names:
        data_grouped[col, 'ci'] = 1.64*data_grouped[col, 'std'] / np.sqrt(data_grouped[col, 'count'])
        data_grouped[col, 'ci_lower'] = data_grouped[col, 'mean'] - data_grouped[col, 'ci']
        data_grouped[col, 'ci_upper'] = data_grouped[col, 'mean'] + data_grouped[col, 'ci']

    for col in column_names:
        x = data_grouped[col].index
        y = data_grouped[col, 'mean']
        ax.plot(x, y, label=col);
        ax.fill_between(
            x, data_grouped[col, 'ci_lower'], data_grouped[col, 'ci_upper'], alpha=.15
        );
        ax.legend();
    return ax

plt.ioff()
ax=plt.gca();

out=widgets.Output()
vbox=widgets.VBox([out])
display(vbox);

data_list = []
max_round = 50
for t in range(max_round):
    simulator = NetworkSimulator(Tx_radius=200, Rx_radius=25, num_Tx_netA=5, num_Rx_per_Tx_netA=8, num_Tx_netB=5, num_Rx_per_Tx_netB=8)

    max_power = 200
    netB_power_training_mode = 'dependent' # ['dependent', 'zero', 'random']
    netB_power_testing_mode = 'dependent' # ['dependent', 'zero', 'random', 'uniform']

    max_iterations = 500

    model_comparison = ExperimentInterferenceModelComparison(simulator=simulator)
    model_comparison.data_generation(num_samples=1000, max_power=max_power, netB_power_mode=netB_power_training_mode)
    model_comparison.modeling(synthetic_constrained=True)
    starting_powers_mW = [1e-5 for i in range(simulator.num_Rx)]
    # Rx_weights = np.random.uniform(0.1, 1, simulator.num_Rx)
    Rx_weights = np.ones(simulator.num_Rx)

    rates_original_method = stochastic_wmmse(
        simulator=simulator,
        Rx_powers_mW=starting_powers_mW,
        Rx_max_powers_mW=max_power,
        Rx_weights=Rx_weights,
        interference_mode='original',
        interference_models=[],
        netB_power_mode = netB_power_testing_mode,
        power_corr_mat=model_comparison.power_corr_mat,
        max_iter=max_iterations,
    );

    rates_local_method = stochastic_wmmse(
        simulator=simulator,
        Rx_powers_mW=starting_powers_mW,
        Rx_max_powers_mW=max_power,
        Rx_weights=Rx_weights,
        interference_mode='local',
        interference_models=[],
        netB_power_mode = netB_power_testing_mode,
        power_corr_mat=model_comparison.power_corr_mat,
        max_iter=max_iterations,
    );

    rates_sc_method = stochastic_wmmse(
        simulator=simulator,
        Rx_powers_mW=starting_powers_mW,
        Rx_max_powers_mW=max_power,
        Rx_weights=Rx_weights,
        interference_mode='sc_estimate',
        interference_models=model_comparison.sc_models,
        netB_power_mode = netB_power_testing_mode,
        power_corr_mat=model_comparison.power_corr_mat,
        max_iter=max_iterations,
    );
    data_slice = pd.DataFrame({
        'iteration': range(len(rates_original_method)), 
        'wmmse': rates_original_method,
        'wmmse_local': rates_local_method,
        'wmmse_sc': rates_sc_method,
    })
    data_list.append(data_slice)
    data = pd.concat(data_list).reset_index(drop=True)

    ax.clear();
    ax = plot_convergence(data, ax);
    with out:
        clear_output(wait=True);
        print(f'Round:{t}')
        display(ax.figure);

In [20]:
data.to_csv(
    f'data/{netB_power_training_mode}_{netB_power_testing_mode}_iter{max_round}_txA{simulator.num_Tx_netA}_txB{simulator.num_Tx_netB}_rx{simulator.num_Rx_per_Tx_netB}.csv',
    index=False
)

In [19]:
f'data/{netB_power_training_mode}_{netB_power_testing_mode}_iter{max_round}_txA{simulator.num_Tx_netA}_txB{simulator.num_Tx_netB}_rx{simulator.num_Rx_per_Tx_netB}.csv',


('data/dependent_dependent_iter50_txA5_txB5_rx8.csv',)